In [1]:
# To fetch the other modules from the main directories 
import sys
sys.path.append('/home/harish/GitRepos/Smart_Helm/')
import pandas as pd
import os
from data_processing.raw_data_parser import transform_gps
from data_processing.raw_data_parser import get_experiment_details
import pyxdf

In [2]:
import dtale

In [2]:
file_csv = "Citipost_2021_09_16_JS_02_location_smarthelm_gps.csv"
file_xdf = "Citipost_2021_09_16_JS_02_raw_eeg_sub-P001_ses-S001_task-Default_run-001_eeg.xdf"
remote_file =  "Citipost/2021/JS/1/3/5/6"

In [4]:
geo_coords = pd.read_csv(file_csv, sep=';')

In [5]:
geo_coords['MSG'][0]

'{"_type":"location","BSSID":"e4:c3:2a:8a:45:57","SSID":"TP-Link_4557","acc":27,"alt":60,"batt":56,"bs":1,"conn":"w","created_at":1631801383,"lat":53.1897265,"lon":8.2282876,"tid":"te","tst":1631801383,"vac":3,"vel":3}'

In [6]:
df = transform_gps(file_csv, "Citipost/2021/JS/1/3/5/6")

In [7]:
df.info()
# here from the df epoch_timestamp_tst and seconds_of_day should be removed
#modified_df = df.drop(columns=['epoch_timestamp_tst', 'seconds_of_day'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   altitude                901 non-null    float64
 1   latitude                901 non-null    float64
 2   longitude               901 non-null    float64
 3   accuracy                901 non-null    float64
 4   velocity                901 non-null    float64
 5   experiment_date         901 non-null    object 
 6   participant_id          901 non-null    object 
 7   experiment_nr           901 non-null    int64  
 8   experiment_location_id  901 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 63.5+ KB


In [24]:
def transform_gps_csv(file_xdf, file_csv, remote_file):
    """
    args: xdf_file
    
    returns: 
    
    """
    data_streams, header = pyxdf.load_xdf(file_xdf)
    for x in data_streams:
        if x['info']['name'] == ['GoProCameraStream']:      
            df_gps = transform_gps(file_csv, remote_file)
            gps_start_lsl = x['time_stamps'][0]  + 80
            gps_time_stamps = []
            for x in range( len(df_gps['latitude'])):
                gps_start_lsl = gps_start_lsl + 1
                gps_time_stamps.append(gps_start_lsl) 
            df_gps['lsl_timestamp'] = gps_time_stamps
    return df_gps

In [17]:
df_gps = transform_gps(file_csv, remote_file )

In [25]:
gps = transform_gps_csv(file_xdf, file_csv, remote_file)
#x = dtale.show(gps)
#df_gps_lsl =  pd.DataFrame(df_gps_dict)

In [27]:
gps.head()

,altitude,latitude,longitude,accuracy,velocity,experiment_date,participant_id,experiment_nr,experiment_location_id,lsl_timestamp
0,60.0,53.189726,8.228288,27.0,3.0,2021-09-16,3,5,Citipost,624169.54861
1,60.0,53.189726,8.228303,28.0,3.0,2021-09-16,3,5,Citipost,624170.54861
2,60.0,53.189726,8.228303,28.0,3.0,2021-09-16,3,5,Citipost,624171.54861
3,60.0,53.189728,8.228319,29.0,3.0,2021-09-16,3,5,Citipost,624172.54861
4,60.0,53.189731,8.228337,32.0,4.0,2021-09-16,3,5,Citipost,624173.54861


In [ ]:
dataset_js_2[2]['time_stamps'][0:10]

In [20]:
data_streams, header = pyxdf.load_xdf(file_xdf)

In [29]:
for x in data_streams:
    if x['info']['name'] == ['GoProCameraStream']:      
        df_gps = transform_gps(file_csv, remote_file )
        start_lst_ts = x['time_stamps'][0] + 80
        print(float(start_lst_ts))
        df_gps['lsl_timestamp'] = list(range(float(start_lst_ts), len(df_gps['latitude']))) 
        #x['time_stamps'][0] = x['time_stamps'][0]
        #print(x['time_stamps'][0])

624168.5486100066


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
for k,v in d.items():  
    local_data_path = download(args.bucket_name, v["raw"])
    etd_df, exp_date = transform_etd(local_data_path, filepath)
    eeg_df = transform_eeg(local_data_path, exp_date,  filepath)
    load_table_to_db(eeg_df, 'eeg_data')
    load_table_to_db(etd_df, 'etd_data')
    gps_df = transform_gps_xdf(local_data_path, exp_date, filepath)
    if len(gps_df) == 1:
        local_data_path_gps = download(args.bucket_name, v["location"])
        try:
            gps_df = transform_gps_csv(local_data_path, local_data_path_gps, v["location"])
        except:
            print("gps data not available")
            
    load_table_to_db(gps_df, 'gps_data')
    fact_df = fact_df_main(eeg_df, etd_df, gps_df)
    load_table_to_db(fact_df, 'fact_data')


         
    
